In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup

import pandas as pd
from datetime import datetime

In [2]:
aylar = {
    "Ocak": "01",
    "Şubat": "02",
    "Mart": "03",
    "Nisan": "04",
    "Mayıs": "05",
    "Haziran": "06",
    "Temmuz": "07",
    "Ağustos": "08",
    "Eylül": "09",
    "Ekim": "10",
    "Kasım": "11",
    "Aralık": "12"
}

In [3]:
def get_details_new(element, cmb):
    year = int(element.find_all("td", {"class": "searchResultsAttributeValue"})[0].text)
    mileage = int(element.find_all("td", {"class": "searchResultsAttributeValue"})[1].text.replace(".", ""))
    color = element.find_all("td", {"class": "searchResultsAttributeValue"})[2].text.strip()
    price = int(element.find("td", {"class": "searchResultsPriceValue"}).text.strip()[:-3].replace(".", ""))
    p1 ,_ ,p3 = element.find("td", {"class": "searchResultsDateValue"}).text.strip().split("\n")
    p1, p2 = p1.split(" ")
    p2 = aylar[p2]
    final_date = datetime.strptime(f"{p1}-{p2}-{p3}", "%d-%m-%Y")
    listing_age = (datetime.now() - final_date).days
    location = element.find("td", {"class": "searchResultsLocationValue"}).get_text(separator=" ").strip().split(" ")[0]
    fuel = cmb[0]
    transmission = cmb[1]

    model = element.find("td", {"class": "searchResultsTagAttributeValue"}).text.strip()
    listing_id = int(element.get_attribute_list("data-id")[0])

    #link = "https://www.sahibinden.com/" + element.find("a", {"class": "classifiedTitle"}).get("href")
    
    return {
        "model": model,
        "age": datetime.now().year - year,
        "mileage": mileage,
        "color": color,
        "price": price,
        "listing_age": listing_age,
        "location": location,
        "fuel": fuel,
        "transmission": transmission,
        "listing_id": listing_id
    }

In [4]:
fuel = ["benzin", "benzin-lpg", "dizel"]
transmission = ["manuel", "otomatik"]

In [5]:
all_combinations = []
for i in fuel:
    for j in transmission:
        all_combinations.append((i, j))

In [6]:
info = []

In [7]:
model_info = ["bmw", "3-serisi", "320i"]
model_name = "-".join(model_info)
number_of_pages = 1

In [8]:
for cmb in all_combinations:
    for i in range(1, number_of_pages+1):
        driver=webdriver.Edge()
        driver.maximize_window()
        link = f"https://www.sahibinden.com/{model_name}/{cmb[0]}/{cmb[1]}?pagingOffset={(i-1)*50}&pagingSize=50"
        driver.get(link)
        print("Directing...", link)

        soup = BeautifulSoup(driver.page_source,'html.parser')
        all_elements = soup.find("tbody", {"class": "searchResultsRowClass"})
        
        if all_elements:
            individual_elements = all_elements.find_all("tr", {"class": "searchResultsItem"})
            
            for item in individual_elements:
                if "classicNativeAd" in item.get_attribute_list("class"):
                    continue
                    # This check prevents the code from crushing on ads.

                info.append(get_details_new(item, cmb))

            driver.quit()
        else:
            driver.quit()
            break

Directing... https://www.sahibinden.com/bmw-3-serisi-320i/benzin/manuel?pagingOffset=0&pagingSize=50
Directing... https://www.sahibinden.com/bmw-3-serisi-320i/benzin/otomatik?pagingOffset=0&pagingSize=50
Directing... https://www.sahibinden.com/bmw-3-serisi-320i/benzin-lpg/manuel?pagingOffset=0&pagingSize=50
Directing... https://www.sahibinden.com/bmw-3-serisi-320i/benzin-lpg/otomatik?pagingOffset=0&pagingSize=50
Directing... https://www.sahibinden.com/bmw-3-serisi-320i/dizel/manuel?pagingOffset=0&pagingSize=50
Directing... https://www.sahibinden.com/bmw-3-serisi-320i/dizel/otomatik?pagingOffset=0&pagingSize=50


In [9]:
data = pd.DataFrame(info)
data.drop_duplicates(["listing_id"], keep="first", inplace=True)
data.drop(["listing_id"], axis=1, inplace=True)

In [11]:
data.to_csv("new_dataset.csv", index=False)

In [10]:
data

,model,age,mileage,color,price,listing_age,location,fuel,transmission
0,320i Cabrio,27,278000,Mor,715000,5,Kırıkkale,benzin,manuel
1,320i,32,337000,Kırmızı,459000,6,İstanbul,benzin,manuel
2,320i,27,239610,Kırmızı,715000,1,Çankırı,benzin,manuel
3,320i,32,150000,Mavi,365000,3,Batman,benzin,manuel
4,320i,26,270000,Füme,440000,1,Gaziantep,benzin,manuel
...,...,...,...,...,...,...,...,...,...
200,320i Premium,17,190000,Füme,850000,2,Kastamonu,benzin-lpg,otomatik
201,320i,26,350000,Mavi,400000,4,Çorum,benzin-lpg,otomatik
202,320i,27,291000,Gri,469000,2,Kayseri,benzin-lpg,otomatik
203,320i Premium,14,246000,Siyah,980000,2,Giresun,benzin-lpg,otomatik
